In [44]:
import cv2
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf
import keras

In [45]:
labels_dict = {0:'back_ground', 
                 1:'A', 
                 2:'B', 
                 3:'C', 
                 4:'D', 
                 5:'E',
                 6:'F',
                 7:'G',
                 8:'H',
                 9:'I',
                 10:'J',
                 11:'K',
                 12:'L',
                 13:'M',
                 14:'N',
                 15:'O',
                 16:'P',
                 17:"Q",
                 18:'R',
                 19:'S',
                 20:'T', 
                 21:'U', 
                 22:'V',
                 23:'W',
                 24:'X',
                 25:'Y',
                 26:'Z'}

In [46]:
model = keras.models.load_model("Model2.h5")

In [47]:
video = cv2.VideoCapture(0)

frame_width = int(video.get(3))
frame_height = int(video.get(4))
   
size = (frame_width, frame_height)
   
#same video on hard drive
result_save = cv2.VideoWriter('Demo.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

while True:
    ret, frame = video.read()
    count=0
    if ret == True:
        
#         frame=cv2.resize(frame,(700,400))
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        count+=1
      
        _ , thresh =cv2.threshold(gray,10,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#         count hands in frame
        contour, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      
        # Check If hand exist in the frame
        if len(contour) == 0:
            pass
        else:
            
            hand_segment = max(contour, key=cv2.contourArea)
            cv2.drawContours(frame, [hand_segment + (300, 20)], -1, (255, 0, 0),1)
            masked = cv2.bitwise_and(frame, frame, mask=thresh)
            
            count=0
            gray=cv2.cvtColor(masked,cv2.COLOR_BGR2GRAY)
            img_=cv2.resize(gray,(64,64))
            img_ = img_.reshape(-1,64 , 64, 1)
            
            result=model.predict(img_)
            result2=np.argmax(result,axis=1)[0]
            char=labels_dict[result2]
            cv2.putText(frame,char,(10,150), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
       
        cv2.imshow("Foreground", masked)
        cv2.imshow("original", frame)
        result_save.write(frame)
        if cv2.waitKey(60) & 0xff == ord('q'):
            break
    else:
        break
cv2.destroyAllWindows()
video.release()
            